# **Solving the Potter's Family Puzzle using AIMA's Logic Library**

In this notebook, we will solve the Potter’s Family Puzzle using two different approaches:

1. **Propositional Logic**
2. **First-Order Logic**

Both approaches will be implemented using the AIMA (Artificial Intelligence: A Modern Approach) logic library.

### *Installing aima library dependencies*

In [ ]:
! pip install networkx
! pip install ipythonblocks
! pip install sortedcontainers

### *Importing all the functions we will use from aima library*

In [6]:
from aima import logic
from aima.logic import FolKB, fol_fc_ask, PropKB, fol_bc_ask, dpll_satisfiable
from aima.utils import expr, Expr

##***The Propositional Logic Approach***

Defining all constants lists

In [7]:
people = ['MummyPotter', 'DaddyPotter', 'Betty', 'Peter', 'AuntPolly']
desserts = ['NapoleonCake', 'Marmalade', 'IceCream', 'Marshmallow', 'Waffles']
dreams = ['Paris', 'Sea', 'SwanLake', 'Coins']
hobbies = ['Yoga', 'Ballet', 'Fishing', 'CoinCollecting', 'Sewing']

Creating rules for family members having only one dessert

In [9]:
rules = []
# Each person (except AuntPolly) must like exactly one dessert
family_members = [p for p in people if p != 'AuntPolly']
for person in family_members:
    statement = ''
    for i in range(len(desserts)):
        statement = statement + f'({person}{desserts[i]}'
        for j in range(len(desserts)):
            if i != j:
                statement = statement + f' & ~{person}{desserts[j]}'
        statement = statement + ') | '
    rules.append(statement[:-2])

# AuntPolly cannot like any desserts
for dessert in desserts:
    rules.append(f'~AuntPolly{dessert}')

Each Person must have exactly one dream

In [10]:
for person in family_members:
        statement = ''
        for i in range(len(dreams)):
            statement = statement + f'({person}{dreams[i]}'
            for j in range(len(dreams)):
                if i != j:
                    statement = statement + f' & ~{person}{dreams[j]}'
            statement = statement + ') | '
        rules.append(statement[:-2])

Facts from the puzzle:
1. Mummy Potter attends yoga classes on Mondays and Thursdays.
2. A person loving ice cream dreams of visiting Paris.
3. Betty likes only marmalade.
4. Mummy eats only marshmallows.
5. The Potters have three money boxes for their dreams at their home: one for a trip to
the sea, one for a ticket to the Swan Lake ballet, one for a new album for the collection of coins.
6. Aunt Polly has a sewing machine and a collection of sewing materials at home. She made a ballet suit for Betty for her classes.
7. Peter often goes fishing with his dad, but he quickly becomes bored of it and begins to
walk down the shore looking for rare coins for his collection.
8. Peter doesn’t like anything with cream.
9. Peter and Betty’s parents have made the same New Year wish both.
10. On holidays, Mummy prepares the family’s favorite desserts: Napoleon cake, marmalade, and waffles.

In [11]:
rules.extend([
        # Rule 1: Mummy Potter does yoga
        'MummyPotterYoga',

        # Rule 2: Ice cream lover dreams of Paris (including AuntPolly)
        '(MummyPotterIceCream ==> MummyPotterParis)',
        '(DaddyPotterIceCream ==> DaddyPotterParis)',
        '(BettyIceCream ==> BettyParis)',
        '(PeterIceCream ==> PeterParis)',
        '(AuntPollyIceCream ==> AuntPollyParis)',

        # Rule 3: Betty likes only marmalade
        'BettyMarmalade & ~BettyNapoleonCake & ~BettyMarshmallow & ~BettyIceCream & ~BettyWaffles',

        # Rule 4: Mummy eats only marshmallows
        'MummyPotterMarshmallow & ~MummyPotterNapoleonCake & ~MummyPotterMarmalade & ~MummyPotterIceCream & ~MummyPotterWaffles',

        # Rule 5: Three specific dreams exist
        f'{" | ".join([f"{p}Sea" for p in family_members])}',
        f'{" | ".join([f"{p}SwanLake" for p in family_members])}',
        f'{" | ".join([f"{p}Coins" for p in family_members])}',

        # Rule 6: Aunt Polly sews and Betty does ballet
        'AuntPollySewing',
        'BettyBallet',

        # Rule 7: Peter fishes and collects coins
        'PeterFishing & PeterCoinCollecting',
        # Peter dreams of coins (implied by coin collecting)
        'PeterCoins',

        # Rule 8: Peter doesn't like cream (Napoleon cake)
        '~PeterNapoleonCake',

        # Rule 9: Parents have same dream
        '(MummyPotterParis <=> DaddyPotterParis)',
        '(MummyPotterSea <=> DaddyPotterSea)',
        '(MummyPotterSwanLake <=> DaddyPotterSwanLake)',
        '(MummyPotterCoins <=> DaddyPotterCoins)',

        # Rule 10: Available desserts must all be assigned
        f'{" | ".join([f"{p}NapoleonCake" for p in family_members])}',
        f'{" | ".join([f"{p}Waffles" for p in family_members])}',
    ])

Initilize the knowledge base and helping query function

In [14]:
# Initialize knowledge base
prop_KB = PropKB()
for rule in rules:
    prop_KB.tell(expr(rule))

# Helping query function
def query_dpll(clauses, query):
    combined = Expr('&', *clauses) & query
    return dpll_satisfiable(combined)

Query for Q1: Who likes the Napoleon cake?

In [17]:
print("Query 1: Who likes Napoleon Cake?")
for person in people:
    query = expr(f'{person}NapoleonCake')
    if query_dpll(prop_KB.clauses, query):
        print(f"{person} likes Napoleon Cake.")

Query 1: Who likes Napoleon Cake?
DaddyPotter likes Napoleon Cake.


Query for Q2: Who dreams of going to Paris?

In [18]:
print("\nQuery 2: Who dreams of going to Paris?")
for person in people:
    query = expr(f'{person}Paris')
    if query_dpll(prop_KB.clauses, query):
        print(f"{person} dreams of going to Paris.")


Query 2: Who dreams of going to Paris?
AuntPolly dreams of going to Paris.


## ***The First-Order Logic Approach***

Initializing the knowledge base

In [19]:
first_order_KB = FolKB()

Constants representing people

In [20]:
first_order_KB.tell(expr("Person(Mummy)"))
first_order_KB.tell(expr("Person(Daddy)"))
first_order_KB.tell(expr("Person(Peter)"))
first_order_KB.tell(expr("Person(Betty)"))
first_order_KB.tell(expr("Person(AuntPolly)"))

Constants representing desserts

In [21]:
first_order_KB.tell(expr("Dessert(IceCream)"))
first_order_KB.tell(expr("Dessert(Marmalade)"))
first_order_KB.tell(expr("Dessert(Marshmallows)"))
first_order_KB.tell(expr("Dessert(NapoleonCake)"))
first_order_KB.tell(expr("Dessert(Waffles)"))
first_order_KB.tell(expr("CreamyFood(NapoleonCake)"))

Facts from the puzzle:
1. Mummy Potter attends yoga classes on Mondays and Thursdays.
2. A person loving ice cream dreams of visiting Paris.
3. Betty likes only marmalade.
4. Mummy eats only marshmallows.
5. The Potters have three money boxes for their dreams at their home: one for a trip to
the sea, one for a ticket to the Swan Lake ballet, one for a new album for the collection of coins.
6. Aunt Polly has a sewing machine and a collection of sewing materials at home. She made a ballet suit for Betty for her classes.
7. Peter often goes fishing with his dad, but he quickly becomes bored of it and begins to
walk down the shore looking for rare coins for his collection.
8. Peter doesn’t like anything with cream.
9. Peter and Betty’s parents have made the same New Year wish both.
10. On holidays, Mummy prepares the family’s favorite desserts: Napoleon cake, marmalade, and waffles.

Initilizing the facts

In [22]:
first_order_KB.tell(expr("Attends(Mummy, Yoga, Monday)"))
first_order_KB.tell(expr("Attends(Mummy, Yoga, Thursday)"))
first_order_KB.tell(expr("Loves(x, IceCream) ==> DreamsOf(x, Paris)"))
first_order_KB.tell(expr("LikesOnly(Betty, Marmalade)"))
first_order_KB.tell(expr("EatsOnly(Mummy, Marshmallows)"))
first_order_KB.tell(expr("HasMoneyBoxFor(Potters, SeaTrip)"))
first_order_KB.tell(expr("HasMoneyBoxFor(Potters, BalletTicket)"))
first_order_KB.tell(expr("HasMoneyBoxFor(Potters, CoinAlbum)"))
first_order_KB.tell(expr("Has(AuntPolly, SewingMachine)"))
first_order_KB.tell(expr("Has(AuntPolly, SewingMaterials)"))
first_order_KB.tell(expr("MadeFor(AuntPolly, BettySuit, Betty)"))
first_order_KB.tell(expr("InterestedIn(Peter, Coins)"))
first_order_KB.tell(expr("CreamyFood(x) ==> Dislike(Peter, x)"))
first_order_KB.tell(expr("Exists(wish, NewYearWish(Mummy, wish) & NewYearWish(Daddy, wish))"))
first_order_KB.tell(expr("Prepares(Mummy, NapoleonCake, Holiday)"))
first_order_KB.tell(expr("Prepares(Mummy, Marmalade, Holiday)"))
first_order_KB.tell(expr("Prepares(Mummy, Waffles, Holiday)"))

Adding Inferences to the knowledge base

In [23]:
first_order_KB.tell(expr("Prepares(Mummy, NapoleonCake, Holiday) & Dislike(Peter, NapoleonCake) & LikesOnly(Betty, Marmalade) & EatsOnly(Mummy, Marshmallows) ==> Likes(Daddy, NapoleonCake)"))
first_order_KB.tell(expr("Hates(Mummy, IceCream)"))
first_order_KB.tell(expr("Hates(Betty, IceCream)"))
first_order_KB.tell(expr("CreamyFood(IceCream) ==> Hates(Peter, IceCream)"))
first_order_KB.tell(expr("Loves(AuntPolly, IceCream)"))
first_order_KB.tell(expr("Loves(AuntPolly, IceCream) ==> DreamsOf(AuntPolly, Paris)"))

Query for Q1: Who likes the Napoleon cake?

In [24]:
print("Question 1: Who likes the Napoleon cake?")
napoleon_cake_lovers = fol_fc_ask(first_order_KB, expr("Likes(p, NapoleonCake)"))
for item in napoleon_cake_lovers:
    print(f"Answer: {item}")

Question 1: Who likes the Napoleon cake?
Answer: {p: Daddy}


Query of Q2: Who dreams of going to Paris?

In [25]:
print("Question 2: Who dreams of going to Paris?")
paris_dreamers = fol_fc_ask(first_order_KB, expr("DreamsOf(p, Paris)"))
for item in paris_dreamers:
    print(f"Answer: {item}")

Question 2: Who dreams of going to Paris?
Answer: {p: AuntPolly}
